In [1]:
import os
import sys
import math
import gc
import random
import joblib
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [128]:
data = joblib.load(os.path.join('..', '..', '0.Data', '05_분석데이터', 'data4WnD.pkl'))
locals().update(data)

In [33]:
sas = joblib.load(os.path.join('..', '..', '0.Data', '05_분석데이터', 'prep4sas.pkl'))

In [14]:
train = pd.read_excel(os.path.join('..', '..', '0.Data', '01_제공데이터', '2020 빅콘테스트 데이터분석분야-챔피언리그_2019년 실적데이터_v1_200818.xlsx'), skiprows = 1)

In [26]:
train['취급액'] = train['취급액'].fillna(-1)

In [30]:
train = train.loc[(train['상품군'] != '무형') & (train['취급액'] != -1)]

In [36]:
train = train.merge(sas['item'], on = '마더코드', how = 'left')

In [51]:
label = np.array([np.zeros(sas['item']['index'].max(), dtype=np.int32) for _ in range(train.shape[0])])

In [55]:
for l, idx in zip(label, train['index'].values):
    l[idx] = 1

In [63]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, LabelEncoder
from keras.layers import Input, Embedding, Dense, Flatten, Dropout, SpatialDropout1D, Activation, concatenate
from keras.optimizers import Adam, SGD
from keras.layers.advanced_activations import ReLU, PReLU, LeakyReLU, ELU
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model
from tensorflow.keras.utils import plot_model

In [170]:
CATEGORICAL_COLUMNS = ['isHoliday', '평일여부', '방송시간대', '계절', '분기']

In [169]:
CONTINUOUS_COLUMNS = ['방송월',
 '방송일',
 '방송시간(시간)',
 '방송시간(분)',
 '합계',
 '컴퓨터 및 주변기기',
 '가전·전자·통신기기',
 '서적',
 '사무·문구',
 '의복',
 '신발',
 '가방',
 '패션용품 및 액세서리',
 '스포츠·레저용품',
 '화장품',
 '아동·유아용품',
 '음·식료품',
 '농축수산물',
 '생활용품',
 '자동차 및 자동차용품',
 '가구',
 '애완용품',
 '여행 및 교통서비스',
 '문화 및 레저서비스',
 'e쿠폰서비스',
 '음식서비스',
 '기타서비스',
 '기타',
 '현재생활형편CSI',
 '현재경기판단CSI',
 '생활형편전망CSI',
 '소비지출전망CSI',
 '주택가격전망CSI',
 '임금수준전망CSI',
 '소비자심리지수',
 '경상지수',
 '불변지수',
 'pca_1',
 'pca_2',
 'pca_3',
 'pca_4',
 'pca_5',
 '강수량(mm)_경기',
 '강수량(mm)_광주',
 '강수량(mm)_대구',
 '강수량(mm)_대전',
 '강수량(mm)_부산',
 '강수량(mm)_서울',
 '강수량(mm)_울산',
 '강수량(mm)_인천',
 '기온(°C)_경기',
 '기온(°C)_광주',
 '기온(°C)_대구',
 '기온(°C)_대전',
 '기온(°C)_부산',
 '기온(°C)_서울',
 '기온(°C)_울산',
 '기온(°C)_인천',
 '습도(%)_경기',
 '습도(%)_광주',
 '습도(%)_대구',
 '습도(%)_대전',
 '습도(%)_부산',
 '습도(%)_서울',
 '습도(%)_울산',
 '습도(%)_인천',
 '시정(10m)_경기',
 '시정(10m)_광주',
 '시정(10m)_대구',
 '시정(10m)_대전',
 '시정(10m)_부산',
 '시정(10m)_서울',
 '시정(10m)_울산',
 '시정(10m)_인천',
 '지면온도(°C)_경기',
 '지면온도(°C)_광주',
 '지면온도(°C)_대구',
 '지면온도(°C)_대전',
 '지면온도(°C)_부산',
 '지면온도(°C)_서울',
 '지면온도(°C)_울산',
 '지면온도(°C)_인천',
 '체감온도_경기',
 '체감온도_광주',
 '체감온도_대구',
 '체감온도_대전',
 '체감온도_부산',
 '체감온도_서울',
 '체감온도_울산',
 '체감온도_인천',
 '풍속(m/s)_경기',
 '풍속(m/s)_광주',
 '풍속(m/s)_대구',
 '풍속(m/s)_대전',
 '풍속(m/s)_부산',
 '풍속(m/s)_서울',
 '풍속(m/s)_울산',
 '풍속(m/s)_인천',
 '최고PM10_경기',
 '최고PM10_광주',
 '최고PM10_대구',
 '최고PM10_대전',
 '최고PM10_부산',
 '최고PM10_서울',
 '최고PM10_울산',
 '최고PM10_인천',
 '최고PM25_경기',
 '최고PM25_광주',
 '최고PM25_대구',
 '최고PM25_대전',
 '최고PM25_부산',
 '최고PM25_서울',
 '최고PM25_울산',
 '최고PM25_인천',
 '최저PM10_경기',
 '최저PM10_광주',
 '최저PM10_대구',
 '최저PM10_대전',
 '최저PM10_부산',
 '최저PM10_서울',
 '최저PM10_울산',
 '최저PM10_인천',
 '최저PM25_경기',
 '최저PM25_광주',
 '최저PM25_대구',
 '최저PM25_대전',
 '최저PM25_부산',
 '최저PM25_서울',
 '최저PM25_울산',
 '최저PM25_인천',
 '평균PM10_경기',
 '평균PM10_광주',
 '평균PM10_대구',
 '평균PM10_대전',
 '평균PM10_부산',
 '평균PM10_서울',
 '평균PM10_울산',
 '평균PM10_인천',
 '평균PM25_경기',
 '평균PM25_광주',
 '평균PM25_대구',
 '평균PM25_대전',
 '평균PM25_부산',
 '평균PM25_서울',
 '평균PM25_울산',
 '평균PM25_인천']

In [171]:
COLUMNS = CONTINUOUS_COLUMNS + CATEGORICAL_COLUMNS

In [198]:
def prep4WnD(train, test):
    
    x_train = train['X']
    train_label = train['label']
    train_size = len(x_train)
    x_test = test['X']
    test_label = test['label']
    
    data = pd.concat([x_train, x_test], axis = 0)
    
    X = data[COLUMNS]
    
    for c in CATEGORICAL_COLUMNS:
        le = LabelEncoder()
        X[c] = le.fit_transform(X[c])
        
    x_train = X.iloc[:train_size]
    y_train = y[:train_size]
    x_test = X.iloc[train_size:]
    y_test = y[train_size:]
    
    x_train_category = np.array(x_train[CATEGORICAL_COLUMNS])
    x_test_category = np.array(x_test[CATEGORICAL_COLUMNS])
    x_train_continue = np.array(x_train[CONTINUOUS_COLUMNS], dtype='float64')
    x_test_continue = np.array(x_test[CONTINUOUS_COLUMNS], dtype='float64')
    
    scaler = StandardScaler()
    X_train_continue_scaler = scaler.fit_transform(x_train_continue)
    X_test_continue = scaler.transform(x_test_continue)
    
    poly = PolynomialFeatures(degree=2, interaction_only=True)
    x_train_category_poly = poly.fit_transform(x_train_category)
    x_test_category_poly = poly.transform(x_test_category)
    
    data4train = (x_train_continue, x_train_category, x_train_category_poly, train_label)
    data4test = (x_test_continue, x_test_category, x_test_category_poly, test_label)

    return data4train, data4test

In [189]:
train = joblib.load(os.path.join('..', '..', '0.Data', '05_분석데이터', 'train_data4WnD.pkl'))
test = joblib.load(os.path.join('..', '..', '0.Data', '05_분석데이터', 'test_data4WnD.pkl'))

In [199]:
a, b = prep4WnD(train, test)

ValueError: could not convert string to float: '아침'

In [139]:
X_category = np.array(X[CATEGORICAL_COLUMNS])
X_continue = np.array(X[CONTINUOUS_COLUMNS])

In [140]:
scaler = StandardScaler()
X_continue_scaler = scaler.fit_transform(X_continue)

In [141]:
poly = PolynomialFeatures(degree=2, interaction_only=True)
X_category_poly = poly.fit_transform(X_category)

In [142]:
def get_deep_model():

    category_inputs = []
    category_embeds = []
    # 카테고리컬 데이터 임베딩
    for i in range(len(CATEGORICAL_COLUMNS)):
        input_i = Input(shape=(1,), dtype='int32')
        dim = len(np.unique(data[CATEGORICAL_COLUMNS[i]]))
        embed_dim = int(np.ceil(dim ** 0.5)) # embedding 차원을 0.5배 정도로 해서 한다.
        embed_i = Embedding(dim, embed_dim, input_length=1)(input_i)
        flatten_i = Flatten()(embed_i)
        category_inputs.append(input_i)
        category_embeds.append(flatten_i)
    # continuous 데이터 input
    continue_input = Input(shape=(len(CONTINUOUS_COLUMNS),))
    continue_dense = Dense(256, use_bias=False)(continue_input)
    # category와 continue를 합침
    concat_embeds = concatenate([continue_dense] + category_embeds)
    concat_embeds = Activation('relu')(concat_embeds)
    bn_concat = BatchNormalization()(concat_embeds)

    fc1 = Dense(512, use_bias=False)(bn_concat)
    relu1 = ReLU()(fc1)
    bn1 = BatchNormalization()(relu1)
    fc2 = Dense(256, use_bias=False)(bn1)
    relu2 = ReLU()(fc2)
    bn2 = BatchNormalization()(relu2)
    fc3 = Dense(128)(bn2)
    relu3 = ReLU()(fc3)
    
    return category_inputs, continue_input, relu3

In [143]:
def get_wide_model():
    dim = X_category_poly.shape[1]
    return Input(shape=(dim,))

In [144]:
category_inputs, continue_input, deep_model = get_deep_model()
wide_model = get_wide_model()

In [146]:
out_layer = concatenate([deep_model, wide_model])
inputs = [continue_input] + category_inputs + [wide_model]
output = Dense(803, activation='softmax')(out_layer)
model = Model(inputs=inputs, outputs=output)

In [147]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, 1)            0                                            
____________________________________________________________________________________________

In [158]:
early_stopping = EarlyStopping(monitor='val_loss', patience=100)

In [159]:
input_data = [X_continue] + [X_category[:, i] for i in range(X_category.shape[1])] + [X_category_poly]

In [160]:
epochs = 100
optimizer ='adam'
batch_size = 128

In [161]:
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(input_data, label, epochs=epochs, batch_size=batch_size, validation_split=0.15, callbacks=[early_stopping])

Train on 30072 samples, validate on 5307 samples
Epoch 1/100
30072/30072 [==============================] - 5s 151us/step - loss: 3.4793 - accuracy: 0.1333 - val_loss: 16.0382 - val_accuracy: 0.0179
Epoch 2/100
30072/30072 [==============================] - 4s 123us/step - loss: 3.3699 - accuracy: 0.1405 - val_loss: 17.4858 - val_accuracy: 0.0168
Epoch 3/100
30072/30072 [==============================] - 4s 125us/step - loss: 3.2974 - accuracy: 0.1493 - val_loss: 14.9847 - val_accuracy: 0.0228
Epoch 4/100
30072/30072 [==============================] - 4s 124us/step - loss: 3.2306 - accuracy: 0.1555 - val_loss: 22.8399 - val_accuracy: 0.0107
Epoch 5/100
30072/30072 [==============================] - 4s 137us/step - loss: 3.1611 - accuracy: 0.1657 - val_loss: 20.7644 - val_accuracy: 0.0313
Epoch 6/100
30072/30072 [==============================] - 4s 132us/step - loss: 3.1046 - accuracy: 0.1683 - val_loss: 23.9003 - val_accuracy: 0.0175
Epoch 7/100
30072/30072 [==========================